# Hyperparameter tuning with XGBoost, Ray Tune, Hyperopt and Optuna

Finding the best hyperparameters for complex modern machine learning algorithms is time-consuming. XGBoost, LightGBM, and neural networks have many tuning parameters and combinations, and a fine-grained grid search may be infeasible.

Fortunately, modern hyperparameter tuning algos, like HyperOpt and Optuna, can run many tests concurrently on a single machine or on a cluster, accelerating the tuning process, saving time and yielding better hyperparameters. This post will demonstrate speeding hyperparameter tuning using Ray Tune, HyperOpt and Optuna locally or on a clusters to significantly accelerate tuning.

Outline:
- Overview of hyperparameter tuning
- Baseline linear regression with no hyperparameters
- ElasticNet with L1 and L2 regularization using ElasticNetCV hyperparameter optimization
- ElasticNet with GridSearchCV hyperparameter optimization
- XGBoost with sequential grid search over hyperparameter subsets with early stopping 
- XGBoost with Ray, HyperOpt and BayesOpt search algorithms
- Accelerate advanced algorithms with a Ray cluster
- Conclusions

But first some results:

| ML Algo           | Hyperparameter search algo   | CV Error (RMSE in $)  | Time     |
|-------------------|------------------------------|-----------------------|----------|
| Linear Regression | None                         | $18192                |   0:01s  |
| ElasticNet        | ElasticNetCV (Grid Search)   | $18122                |   0:02s  |          
| ElasticNet        | GridSearchCV                 | $18061                |   0:05s  |          
| XGB               | Sequential Grid Search       | $18783                |   36:09  |
| XGB               | HyperOpt (128 samples)       | $18808                |   21:41  |
| XGB               | BayesOpt                     | $18506                | 1:15:04  |
| XGB               | Optuna                       | $18618

These are the main approaches to hyperparameter tuning https://en.wikipedia.org/wiki/Hyperparameter_optimization

- Grid search: given a finite set of discrete values for each hyperparameter, exhaustively cross-validate all combinations

- Random search: given a discrete or continuous distribution for each hyperparameter, randomly sample from the joint distribution. Generally [more efficient than exhaustive grid search.](https://dl.acm.org/doi/10.5555/2188385.2188395 ) 

- Bayesian optimization: update the search space as you search based on outcomes of prior searches

- Gradient-based optimization: attempt to estimate the gradient of the CV metric with respect to the hyperparameter and ascend/descend the gradient.

- Evolutionary optimization: sample the search space, discard combinations with poor metrics, and genetically evolve new combinations to try based on the successful combinations.

- Population-based: A method of performing hyperparameter optimization at the same time as training.

We're going to implement Bayesian optimization with HyperOpt and Optuna. When we perform a grid search, we can consider the search space a prior belief that the all the combinations have equal probability of being the best combination. Perhaps after an initial search on a broad, coarsely spaced grid, we might do a deeper dive in a smaller area around the best metric with a fine-spaced grid. In Bayesian terms we updated our prior belief.

Bayesian optimization first samples randomly eg 30 combinations and computes the CV metric for each combination. Then it updates the distribution it samples from, to be more likely to sample combinations near the good metrics, and less likely to sample combinations near the poor metrics. As it continues to sample, it continues to update the search distribution based on the metrics it finds.

Early stopping is highly useful: often we can discard a combination without fully training it.

In this post we will use data from the Ames Housing Dataset https://www.kaggle.com/c/house-prices-advanced-regression-techniques . The original data has 79 raw features. The data we will use has 100 features with a fair amount of feature engineering from [my own attempt at modeling](https://github.com/druce/iowa), which was in the top 5% or so when I submitted it.

https://arxiv.org/abs/1502.02127
https://link.springer.com/chapter/10.1007/978-3-030-05318-5_1
https://arxiv.org/abs/2003.05689




In [24]:
from itertools import product
from datetime import datetime, timedelta
import os
import random
import string

import numpy as np
import pandas as pd

import sklearn
from sklearn.linear_model import LinearRegression, ElasticNet, ElasticNetCV, Ridge, RidgeCV
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV, KFold
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import make_pipeline

#!conda install -y -c conda-forge  xgboost 
import xgboost
from xgboost import XGBRegressor
from xgboost import plot_importance

import lightgbm
from lightgbm import LGBMRegressor

import ray
from ray import tune
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.bayesopt import BayesOptSearch
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.suggest.optuna import OptunaSearch
from ray.tune.logger import DEFAULT_LOGGERS
from ray.tune.integration.wandb import WandbLogger

# pip install hyperopt
# pip install optuna

import wandb
os.environ['WANDB_NOTEBOOK_NAME']='hyperparameter_optimization.ipynb'

print(datetime.now())

print ("%-20s %s"% ("numpy", np.__version__))
print ("%-20s %s"% ("pandas", pd.__version__))
print ("%-20s %s"% ("sklearn", sklearn.__version__))
print ("%-20s %s"% ("xgboost", xgboost.__version__))
print ("%-20s %s"% ("lightgbm", lightgbm.__version__))
print ("%-20s %s"% ("ray", ray.__version__))


2020-10-12 22:41:05.803443
numpy                1.19.1
pandas               1.1.3
sklearn              0.23.2
xgboost              1.2.0
lightgbm             2.3.0
ray                  1.0.0


In [2]:
# set seed for reproducibility
RANDOMSTATE = 42
np.random.seed(RANDOMSTATE)


In [3]:
def get_random_tag(length):
    """random tag for experiments"""
    letters_and_digits = string.ascii_letters + string.digits
    result_str = ''.join((random.choice(letters_and_digits) for i in range(length)))
    return result_str.upper()

get_random_tag(8)

'DNUYW1YC'

In [4]:
# import train data
df = pd.read_pickle('df_train.pickle')

response = 'SalePrice'
predictors = ['YearBuilt',
              'BsmtFullBath',
              'FullBath',
              'KitchenAbvGr',
              'GarageYrBlt',
              'LotFrontage',
              'MasVnrArea',
              '1stFlrSF',
              'GrLivArea',
              'GarageArea',
              'WoodDeckSF',
              'PorchSF',
              'AvgBltRemod',
              'FireBathRatio',
              'TotalSF x OverallQual x OverallCond',
              'AvgBltRemod x Functional x TotalFinSF',
              'Functional x OverallQual',
              'KitchenAbvGr x KitchenQual',
              'GarageCars x GarageYrBlt',
              'GarageQual x GarageCond x GarageCars',
              'HeatingQC x Heating',
              'monthnum',
              'log_YearBuilt',
              'log_LotArea',
              'log_TotalFinSF',
              'log_GarageRatio',
              'log_TotalSF x OverallQual x OverallCond',
              'log_TotalSF x OverallCond',
              'log_AvgBltRemod x TotalFinSF',
              'sq_2ndFlrSF',
              'sq_BsmtFinSF',
              'sq_BsmtFinSF x BsmtQual',
              'sq_BsmtFinSF x BsmtBath',
              'BldgType_4',
              'BsmtExposure_1',
              'BsmtExposure_4',
              'BsmtFinType1_1',
              'BsmtFinType1_2',
              'BsmtFinType1_4',
              'BsmtFinType1_5',
              'BsmtFinType1_6',
              'CentralAir_0',
              'CentralAir_1',
              'Condition1_1',
              'Condition1_3',
              'ExterCond_2',
              'ExterQual_2',
              'Exterior1st_4',
              'Exterior1st_5',
              'Exterior1st_10',
              'Fence_0',
              'Fence_2',
              'Foundation_1',
              'Foundation_5',
              'GarageCars_1',
              'GarageFinish_2',
              'GarageFinish_3',
              'GarageType_2',
              'HouseStyle_2',
              'KitchenQual_4',
              'LotConfig_0',
              'LotConfig_4',
              'MSSubClass_30',
              'MSSubClass_70',
              'MSZoning_0',
              'MSZoning_1',
              'MSZoning_4',
              'MasVnrType_2',
              'MasVnrType_3',
              'MoSold_1',
              'MoSold_5',
              'MoSold_6',
              'MoSold_11',
              'Neighborhood_3',
              'Neighborhood_4',
              'Neighborhood_5',
              'Neighborhood_10',
              'Neighborhood_11',
              'Neighborhood_16',
              'Neighborhood_17',
              'Neighborhood_19',
              'Neighborhood_22',
              'Neighborhood_24',
              'OverallCond_7',
              'OverallQual_5',
              'OverallQual_6',
              'OverallQual_7',
              'OverallQual_9',
              'PavedDrive_0',
              'PavedDrive_2',
              'SaleCondition_1',
              'SaleCondition_2',
              'SaleCondition_5',
              'SaleType_4',
              'BedroomAbvGr_1',
              'BedroomAbvGr_4',
              'BedroomAbvGr_5',
              'HalfBath_1',
              'TotalBath_1.0',
              'TotalBath_2.5']

X_train, X_test, y_train, y_test = train_test_split(df, df[response], test_size=.25)

display(df[predictors].head())
display(df[[response]].head())


,YearBuilt,BsmtFullBath,FullBath,KitchenAbvGr,GarageYrBlt,LotFrontage,MasVnrArea,1stFlrSF,GrLivArea,GarageArea,...,SaleCondition_1,SaleCondition_2,SaleCondition_5,SaleType_4,BedroomAbvGr_1,BedroomAbvGr_4,BedroomAbvGr_5,HalfBath_1,TotalBath_1.0,TotalBath_2.5
Id,,,,,,,,,,,,,,,,,,,,,
1,7,1,2,1,7,65.0,196.0,856,1710,548.0,...,0,0,0,1,0,0,0,1,0,0
2,34,0,2,1,34,80.0,0.0,1262,1262,460.0,...,0,0,0,1,0,0,0,0,0,1
3,9,1,2,1,9,68.0,162.0,920,1786,608.0,...,0,0,0,1,0,0,0,1,0,0
4,95,1,1,1,12,60.0,0.0,961,1717,642.0,...,1,0,0,1,0,0,0,0,0,0
5,10,1,2,1,10,84.0,350.0,1145,2198,836.0,...,0,0,0,1,0,1,0,1,0,0


,SalePrice
Id,
1,12.247699
2,12.109016
3,12.317171
4,11.849405
5,12.429220


In [5]:
# we are training on a response which is the log of 1 + the sale price
# transform prediction back to original basis with expm1 and evaluate vs. original

def evaluate(y_train, y_pred_train, y_test, y_pred_test):
    """evaluate in train_test split"""
    print('Train RMSE', np.sqrt(mean_squared_error(np.expm1(y_train), np.expm1(y_pred_train))))
    print('Train R-squared', r2_score(np.expm1(y_train), np.expm1(y_pred_train)))
    print('Train MAE', mean_absolute_error(np.expm1(y_train), np.expm1(y_pred_train)))
    print()
    print('Test RMSE', np.sqrt(mean_squared_error(np.expm1(y_test), np.expm1(y_pred_test))))
    print('Test R-squared', r2_score(np.expm1(y_test), np.expm1(y_pred_test)))
    print('Test MAE', mean_absolute_error(np.expm1(y_test), np.expm1(y_pred_test)))

MEAN_RESPONSE=df[response].mean()
def cv_to_raw(cv_val):
    """convert log1p rmse to underlying SalePrice error"""
    return np.expm1(MEAN_RESPONSE+cv_val) - np.expm1(MEAN_RESPONSE)

In [6]:
# always use same k-folds for reproducibility
kfolds = KFold(n_splits=10, shuffle=True, random_state=RANDOMSTATE)


## Baseline linear regression
- Raw CV RMSE 18191.9791
- Wall time 2.81 s

In [7]:
%%time
# Tune lr search space for alphas and l1_ratio
print("LinearRegression")

print(len(predictors), "predictors")

lr = LinearRegression()

#train and evaluate in train/test split
lr.fit(X_train[predictors], y_train)

y_pred_train = lr.predict(X_train[predictors])
y_pred_test = lr.predict(X_test[predictors])
evaluate(y_train, y_pred_train, y_test, y_pred_test)

# evaluate using kfolds, same process as train/test split but average results over 10 folds
# more sample-efficient, less CPU-efficient

scores = -cross_val_score(lr, df[predictors], df[response],
                          scoring="neg_root_mean_squared_error",
                          cv=kfolds,
                          n_jobs=-1)
raw_scores = [cv_to_raw(x) for x in scores]
print()
print("Log1p CV RMSE %.04f (STD %.04f)" % (np.mean(scores), np.std(scores)))
print("Raw CV RMSE %.04f (STD %.04f)" % (np.mean(raw_scores), np.std(raw_scores)))


LinearRegression
100 predictors
Train RMSE 16551.16362165147
Train R-squared 0.9554490130823184
Train MAE 10998.860388695646

Test RMSE 17846.17192179596
Test R-squared 0.9364301769277958
Test MAE 12755.395673639627

Log1p CV RMSE 0.1037 (STD 0.0099)
Raw CV RMSE 18191.9791 (STD 1838.6678)
CPU times: user 204 ms, sys: 346 ms, total: 549 ms
Wall time: 1.48 s


## Native Sklearn xxxCV
- LogisticRegressionCV, LassoCV, RidgeCV, ElasticNetCV, etc.
- Test many hyperparameters in parallel with multithreading
- Note improvement vs. LinearRegression due to controlling overfitting
- RMSE $18103
- Time 5s


In [8]:
%%time
# Tune elasticnet search space for alphas and L1_ratio
# predictor selection used to create the training set used lasso
# so l1 parameter is close to 0
# could use ridge (eg elasticnet with 0 L1 regularization)
# but then only 1 param, more general and useful to do this with elasticnet
print("ElasticnetCV")

# make pipeline
# with regularization must scale predictors
elasticnetcv = make_pipeline(RobustScaler(),
                             ElasticNetCV(max_iter=100000, 
                                          l1_ratio=[0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99],
                                          alphas=np.logspace(-4, -2, 9),
                                          cv=kfolds,
                                          n_jobs=-1,
                                          verbose=1,
                                         ))

#train and evaluate in train/test split
elasticnetcv.fit(X_train[predictors], y_train)

y_pred_train = elasticnetcv.predict(X_train[predictors])
y_pred_test = elasticnetcv.predict(X_test[predictors])
evaluate(y_train, y_pred_train, y_test, y_pred_test)
l1_ratio = elasticnetcv._final_estimator.l1_ratio_
alpha = elasticnetcv._final_estimator.alpha_
print('l1_ratio', l1_ratio)
print('alpha', alpha)

# evaluate using kfolds on full dataset
# I don't see API to get CV error from elasticnetcv, so we use cross_val_score
elasticnet = ElasticNet(alpha=alpha,
                        l1_ratio=l1_ratio,
                        max_iter=10000)

scores = -cross_val_score(elasticnet, df[predictors], df[response],
                          scoring="neg_root_mean_squared_error",
                          cv=kfolds,
                          n_jobs=-1)
raw_scores = [cv_to_raw(x) for x in scores]
print()
print("Log1p CV RMSE %.04f (STD %.04f)" % (np.mean(scores), np.std(scores)))
print("Raw CV RMSE %.04f (STD %.04f)" % (np.mean(raw_scores), np.std(raw_scores)))


ElasticnetCV


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
............................................................................................................................................................................................................................................................................[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.1s
............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

Train RMSE 16782.09907754634
Train R-squared 0.954197115772813
Train MAE 11025.896226946616

Test RMSE 17457.333905669522
Test R-squared 0.9391701570474736
Test MAE 12389.820674779756
l1_ratio 0.01
alpha 0.005623413251903491

Log1p CV RMSE 0.1033 (STD 0.0112)
Raw CV RMSE 18122.0127 (STD 2074.9545)
CPU times: user 4.49 s, sys: 2.19 s, total: 6.68 s
Wall time: 1.41 s


## GridSearchCV
- Useful for algos with no native multithreaded xxxCV
- Test many hyperparameter combinations in parallel with multithreading
- Similar result vs ElasticNetCV, not exact, need more research as to why


In [9]:
%%time
gs = make_pipeline(RobustScaler(),
                   GridSearchCV(ElasticNet(max_iter=100000),
                                param_grid={'l1_ratio': [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99],
                                            'alpha': np.logspace(-4, -2, 9),
                                           },
                                scoring='neg_mean_squared_error',
                                refit=True,
                                cv=kfolds,
                                n_jobs=-1,
                                verbose=1
                               ))

# do cv using kfolds on full dataset
print("\nCV on full dataset")
gs.fit(df[predictors], df[response])
print('best params', gs._final_estimator.best_params_)
print('best score', -gs._final_estimator.best_score_)
l1_ratio = gs._final_estimator.best_params_['l1_ratio']
alpha = gs._final_estimator.best_params_['alpha']

elasticnet = ElasticNet(alpha=alpha,
                        l1_ratio=l1_ratio,
                        max_iter=100000)
print(elasticnet)

scores = -cross_val_score(elasticnet, df[predictors], df[response],
                          scoring="neg_root_mean_squared_error",
                          cv=kfolds,
                          n_jobs=-1)
raw_scores = [cv_to_raw(x) for x in scores]
print()
print("Log1p CV RMSE %.06f (STD %.04f)" % (np.mean(scores), np.std(scores)))
print("Raw CV RMSE %.06f (STD %.04f)" % (np.mean(raw_scores), np.std(raw_scores)))

# difference in average CV scores reported by GridSearchCV and cross_val_score
# with same alpha, l1_ratio, kfolds
# one reason could be that we used simple average, GridSearchCV is weighted by # of samples per fold?
nsamples = [len(z[1]) for z in kfolds.split(df)]
print("weighted average %.06f" % np.average(scores, weights=nsamples))
# not sure why, also ElasticSearchCV shows fewer fits, takes less time



CV on full dataset
Fitting 10 folds for each of 117 candidates, totalling 1170 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 306 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 808 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 1139 out of 1170 | elapsed:    4.5s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done 1170 out of 1170 | elapsed:    4.6s finished


best params {'alpha': 0.0031622776601683794, 'l1_ratio': 0.01}
best score 0.010637685614240404
ElasticNet(alpha=0.0031622776601683794, l1_ratio=0.01, max_iter=100000)

Log1p CV RMSE 0.103003 (STD 0.0109)
Raw CV RMSE 18060.902698 (STD 2008.2407)
weighted average 0.103023
CPU times: user 1.96 s, sys: 571 ms, total: 2.53 s
Wall time: 4.9 s


In [10]:
# roll-our-own CV 
# matches cross_val_score
alpha = 0.0031622776601683794
l1_ratio = 0.01
regressor = ElasticNet(alpha=alpha,
                       l1_ratio=l1_ratio,
                       max_iter=10000)
print(regressor)
cverrors = []
for train_fold, cv_fold in kfolds.split(df): 
    fold_X_train=df[predictors].values[train_fold]
    fold_y_train=df[response].values[train_fold]
    fold_X_test=df[predictors].values[cv_fold]
    fold_y_test=df[response].values[cv_fold]
    regressor.fit(fold_X_train, fold_y_train)
    y_pred_test=regressor.predict(fold_X_test)
    cverrors.append(np.sqrt(mean_squared_error(fold_y_test, y_pred_test)))
    
print("%.06f" % np.average(cverrors))
    

ElasticNet(alpha=0.0031622776601683794, l1_ratio=0.01, max_iter=10000)
0.103003


## XGBoost Sequential Grid Search CV 
- XGBoost has native multithreading, CV
- XGBoost has many tuning parameters so a complete grid search has an unreasonable number of combinations
- We tune reduced sets sequentially and use early stopping. 

### Tuning methodology
- Set an initial set of starting parameters
- Do 10-fold CV
- Use early stopping to halt training in each fold if no improvement after eg 100 rounds, pick hyperparameters to minimize average error over kfolds
- Tune sequentially on groups of hyperparameters that don't interact too much between groups to reduce combinations
- Tune max_depth and min_child_weight 
- Tune subsample and colsample_bytree
- Tune alpha, lambda and gamma (regularization)
- Tune learning rate: lower learning rate will need more rounds/n_estimators
- Retrain on full dataset with best learning rate and best n_estimators (average stopping point over kfolds)

### Notes
- It doesn't seem possible to get XGBoost early stopping and also use GridSearchCV. GridSearchCV doesn't pass the kfolds in a way that XGboost understands for early stopping
- 2 alternative approaches 
    - use native xgboost .cv which understands early stopping but doesn't use sklearn API (uses DMatrix, not np array or dataframe)
    - use sklearn API and roll our own grid search instead of GridSearchCV (used below)
- XGboost terminology differs from sklearn
    - boost_rounds = n_estimators
    - eta = learning_rate
- parameter reference: https://xgboost.readthedocs.io/en/latest/parameter.html
- training reference: https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.training
- times are wall times on an amazon t2.2xlarge instance  
- to set up environment:
    - `conda create --name hyperparam python=3.8`
    - `conda activate hyperparam`
    - `conda install jupyter`
    - `pip install -r requirements.txt`
- round 1 Wall time: 6min 23s
- round 2 Wall time: 19min 22s
- round 3 Wall time: 5min 30s
- round 4 Wall time: 4min 54s
- total time 36:09
- RMSE 18783.117031

In [18]:
#%%time
# this cell runs a single round
# the full process is 
# set initial XGboost parameters
# remove overrides (search for TODO: in this cell)
# run round 1 and override initial max_depth, min_child_weight based on best values (search for TODO:)
# run round 2 and override subsample and colsample_bytree based on best values
# run round 3 and override reg_alpha, reg_lambda, reg_gamma based on best values
# run round 4 and obtain learning_rate and best n_iterations
# this is not an exhaustive list but a representative list of most important parameters to tune
# see https://xgboost.readthedocs.io/en/latest/parameter.html for all parameters

# in XGBoost > 1.0.2 this seems to give a warning
# Parameters: { early_stopping_rounds } might not be used.
# but early stopping seems to be used correctly

max_depth = 5
min_child_weight=5
colsample_bytree = 0.5
subsample = 0.5
reg_alpha = 1e-05
reg_lambda = 1
reg_gamma = 0
learning_rate = 0.01

BOOST_ROUNDS=50000   # we use early stopping so make this arbitrarily high
EARLY_STOPPING_ROUNDS=100 # stop if no improvement after 100 rounds

# round 1: tune depth and min_child_weight
max_depths = list(range(1,5))
min_child_weights = list(range(1,5))
gridsearch_params_1 = product(max_depths, min_child_weights)

# round 2: tune subsample and colsample_bytree
subsamples = np.linspace(0.1, 1.0, 10)
colsample_bytrees = np.linspace(0.1, 1.0, 10)
gridsearch_params_2 = product(subsamples, colsample_bytrees)

# round 2 (refined): tune subsample and colsample_bytree
subsamples = np.linspace(0.4, 0.8, 9)
colsample_bytrees = np.linspace(0.05, 0.25, 5)
gridsearch_params_2 = product(subsamples, colsample_bytrees)

# round 3: tune alpha, lambda, gamma
reg_alphas = np.logspace(-3, -2, 3)
reg_lambdas = np.logspace(-2, 1, 4)
reg_gammas = [0]
#reg_gammas = np.linspace(0, 5, 6)
gridsearch_params_3 = product(reg_alphas, reg_lambdas, reg_gammas)

# round 4: learning rate
learning_rates = reversed(np.logspace(-3, -1, 5).tolist())
gridsearch_params_4 = learning_rates

# TODO: remove these overrides to reset the search
# override initial parameters after search
# round 1:
max_depth=2
min_child_weight=2
# # round 2:
subsample=0.60
colsample_bytree=0.05
# # round 3:  
reg_alpha = 0.003162
reg_lambda = 0.1
reg_gamma = 0

def my_cv(df, predictors, response, kfolds, regressor, verbose=False):
    """Roll our own CV over kfolds with early stopping"""
    metrics = []
    best_iterations = []

    for train_fold, cv_fold in kfolds.split(df): 
        fold_X_train=df[predictors].values[train_fold]
        fold_y_train=df[response].values[train_fold]
        fold_X_test=df[predictors].values[cv_fold]
        fold_y_test=df[response].values[cv_fold]
        regressor.fit(fold_X_train, fold_y_train,
                      early_stopping_rounds=EARLY_STOPPING_ROUNDS,
                      eval_set=[(fold_X_test, fold_y_test)],
                      eval_metric='rmse',
                      verbose=verbose
                     )
        y_pred_test=regressor.predict(fold_X_test)
        metrics.append(np.sqrt(mean_squared_error(fold_y_test, y_pred_test)))
        best_iterations.append(xgb.best_iteration)
    return np.average(metrics), np.std(metrics), np.average(best_iterations)

results = []
best_iterations = []

# TODO: iteratively uncomment 1 of the following 4 lines
# for i, (max_depth, min_child_weight) in enumerate(gridsearch_params_1): # round 1
# for i, (subsample, colsample_bytree) in enumerate(gridsearch_params_2): # round 2
# for i, (reg_alpha, reg_lambda, reg_gamma) in enumerate(gridsearch_params_3): # round 3
for i, learning_rate in enumerate(gridsearch_params_4): # round 4

    params = {
        'max_depth': max_depth,
        'min_child_weight': min_child_weight,
        'subsample': subsample,
        'colsample_bytree': colsample_bytree,
        'reg_alpha': reg_alpha,
        'reg_lambda': reg_lambda,
        'gamma': reg_gamma,
        'learning_rate': learning_rate,
    }
    print("%s params  %3d: %s" % (datetime.strftime(datetime.now(), "%T"), i, params))
    xgb = XGBRegressor(
        objective='reg:squarederror',
        n_estimators=BOOST_ROUNDS,
        early_stopping_rounds=EARLY_STOPPING_ROUNDS,
        random_state=RANDOMSTATE,    
        verbosity=1,
        n_jobs=-1,
        booster='gbtree',   
        base_score=0.5, 
        scale_pos_weight=1,      
        **params
    )
    
    metric_rmse, metric_std, best_iteration = my_cv(df, predictors, response, kfolds, xgb, verbose=False)    
    results.append([max_depth, min_child_weight, subsample, colsample_bytree, reg_alpha, reg_lambda, reg_gamma, 
                   learning_rate, metric_rmse, metric_std, best_iteration])
    
    print("%s %3d result mean: %.6f std: %.6f, iter: %.2f" % (datetime.strftime(datetime.now(), "%T"), i, metric_rmse, metric_std, best_iteration))


results_df = pd.DataFrame(results, columns=['max_depth', 'min_child_weight', 'subsample', 'colsample_bytree', 
                               'reg_alpha', 'reg_lambda', 'reg_gamma', 'learning_rate', 'rmse', 'std', 'best_iter']).sort_values('rmse')
results_df


22:00:35 params    0: {'max_depth': 2, 'min_child_weight': 2, 'subsample': 0.6, 'colsample_bytree': 0.05, 'reg_alpha': 0.003162, 'reg_lambda': 0.1, 'gamma': 0, 'learning_rate': 0.1}
[22:00:35] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { early_stopping_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[22:00:35] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { early_stopping_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[22:00:36] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Pa

[22:00:49] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { early_stopping_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[22:00:51] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { early_stopping_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[22:00:52] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { early_stopping_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or so

[22:02:44] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { early_stopping_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[22:03:00] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { early_stopping_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[22:03:10] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { early_stopping_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or so

,max_depth,min_child_weight,subsample,colsample_bytree,reg_alpha,reg_lambda,reg_gamma,learning_rate,rmse,std,best_iter
2,2,2,0.6,0.05,0.003162,0.1,0,0.010000,0.106412,0.012678,2753.4
1,2,2,0.6,0.05,0.003162,0.1,0,0.031623,0.107067,0.013565,1086.4
3,2,2,0.6,0.05,0.003162,0.1,0,0.003162,0.107130,0.013006,7161.0
4,2,2,0.6,0.05,0.003162,0.1,0,0.001000,0.107471,0.013157,20839.1
0,2,2,0.6,0.05,0.003162,0.1,0,0.100000,0.109695,0.012934,466.3


In [19]:
max_depth = int(results_df.iloc[0]['max_depth'])
min_child_weight = results_df.iloc[0]['min_child_weight']
subsample = results_df.iloc[0]['subsample']
colsample_bytree = results_df.iloc[0]['colsample_bytree']
reg_alpha = results_df.iloc[0]['reg_alpha']
reg_lambda = results_df.iloc[0]['reg_lambda']
reg_gamma = results_df.iloc[0]['reg_gamma']
learning_rate = results_df.iloc[0]['learning_rate']
N_ESTIMATORS = int(results_df.iloc[0]['best_iter'])

params = {
    'max_depth': int(max_depth),
    'min_child_weight': min_child_weight,
    'subsample': subsample,
    'colsample_bytree': colsample_bytree,
    'reg_alpha': reg_alpha,
    'reg_lambda': reg_lambda,
    'gamma': reg_gamma,
    'learning_rate': learning_rate,
    'n_estimators': N_ESTIMATORS,    
}

print(params)

{'max_depth': 2, 'min_child_weight': 2.0, 'subsample': 0.6, 'colsample_bytree': 0.05, 'reg_alpha': 0.003162, 'reg_lambda': 0.1, 'gamma': 0.0, 'learning_rate': 0.01, 'n_estimators': 2753}


In [20]:
%%time
# evaluate without early stopping

xgb = XGBRegressor(
    objective='reg:squarederror',
    random_state=RANDOMSTATE,    
    verbosity=1,
    n_jobs=-1,
    booster='gbtree',   
    base_score=0.5, 
    scale_pos_weight=1        
    **params
)
print(xgb)

scores = -cross_val_score(xgb, df[predictors], df[response],
                          scoring="neg_root_mean_squared_error",
                          cv=kfolds,
                          n_jobs=-1)
raw_scores = [cv_to_raw(x) for x in scores]
print()
print("Log1p CV RMSE %.06f (STD %.04f)" % (np.mean(scores), np.std(scores)))
print("Raw CV RMSE %.06f (STD %.04f)" % (np.mean(raw_scores), np.std(raw_scores)))


TypeError: unsupported operand type(s) for ** or pow(): 'int' and 'dict'

In [21]:
# refactor for ray.tune
def my_xgb(config):
    
    # fix these configs 
    config['max_depth'] += 2   # hyperopt needs left to start at 0 but we want to start at 2
    config['max_depth'] = int(config['max_depth'])
    config['n_estimators'] = int(config['n_estimators'])   # pass float eg loguniform distribution, use int
    
    xgb = XGBRegressor(
        objective='reg:squarederror',
        n_jobs=1,
        random_state=RANDOMSTATE,
        booster='gbtree',   
        base_score=0.5, 
        scale_pos_weight=1, 
        **config,
    )
    scores = np.sqrt(-cross_val_score(xgb, df[predictors], df[response],
                                      scoring="neg_mean_squared_error",
                                      cv=kfolds))
    tune.report(mse=np.mean(scores))
    return {'mse': np.mean(scores)}


In [22]:
config = {
    'max_depth': max_depth-2,
    'min_child_weight': min_child_weight,
    'subsample': subsample,
    'colsample_bytree': colsample_bytree,
    'reg_alpha': reg_alpha,
    'reg_lambda': reg_lambda,
    'gamma': reg_gamma,
    'learning_rate': learning_rate,
    'n_estimators': N_ESTIMATORS,
}

xgb = my_xgb(config)

print(xgb)


Session not detected. You should not be calling this function outside `tune.run` or while using the class API. 


{'mse': 0.10694561357985297}


## HyperOpt
https://conference.scipy.org/proceedings/scipy2013/pdfs/bergstra_hyperopt.pdf
https://github.com/hyperopt/hyperopt
http://hyperopt.github.io/hyperopt/
https://blog.dominodatalab.com/hyperopt-bayesian-hyperparameter-optimization/
    

In [23]:
NUM_SAMPLES=256

start_time = datetime.now()
print("%-20s %s" % ("Start Time", start_time))

algo = HyperOptSearch(random_state_seed=RANDOMSTATE)
# uncomment and set max_concurrent to limit number of cores
# algo = ConcurrencyLimiter(algo, max_concurrent=10)
scheduler = AsyncHyperBandScheduler()

tune_kwargs = {
    "num_samples": NUM_SAMPLES,
    "config": {
        "n_estimators": tune.loguniform(100, 10000),
        "max_depth": tune.randint(0, 6),
        'min_child_weight': tune.randint(0, 6),
        "subsample": tune.quniform(0.4, 0.9, 0.05),
        "colsample_bytree": tune.quniform(0.05, 0.8, 0.05),
        "reg_alpha": tune.loguniform(1e-04, 1),
        "reg_lambda": tune.loguniform(1e-04, 100),
        "gamma": 0,
        "learning_rate": tune.loguniform(0.001, 0.1),
        "wandb": {
            "project": "iowa",
            "api_key_file": "secrets/wandb.txt",
            "log_config": True
        }    
    }
}

analysis = tune.run(my_xgb,
                    name="xgb_hyperopt",
                    metric="mse",
                    mode="min",
                    search_alg=algo,
                    scheduler=scheduler,
                    verbose=1,
                    loggers=DEFAULT_LOGGERS + (WandbLogger, ),
                    **tune_kwargs)

end_time = datetime.now()
print("%-20s %s" % ("Start Time", start_time))
print("%-20s %s" % ("End Time", end_time))
print(str(timedelta(seconds=(end_time-start_time).seconds)))

Trial name,status,loc,colsample_bytree,gamma,learning_rate,max_depth,min_child_weight,n_estimators,reg_alpha,reg_lambda,subsample,wandb/api_key_file,wandb/log_config,wandb/project
my_xgb_851c46f8,ERROR,,0.7,0,0.0105391,0,4,435.406,0.000188888,9.90358,0.65,secrets/wandb.txt,True,iowa
my_xgb_8523af6a,ERROR,,0.3,0,0.00673344,3,2,428.153,0.538855,2.32463,0.6,secrets/wandb.txt,True,iowa
my_xgb_85286f14,ERROR,,0.15,0,0.00133237,5,4,7988.18,0.00163546,19.9476,0.8,secrets/wandb.txt,True,iowa
my_xgb_852d4e3a,ERROR,,0.2,0,0.0187659,2,3,735.64,0.000262686,32.5338,0.6,secrets/wandb.txt,True,iowa
my_xgb_8531d554,ERROR,,0.6,0,0.00155207,0,3,4032.08,0.0246711,0.00107456,0.8,secrets/wandb.txt,True,iowa
my_xgb_8534f112,ERROR,,0.15,0,0.00106609,1,5,2700.39,0.0127785,0.0424791,0.8,secrets/wandb.txt,True,iowa
my_xgb_8537cbf8,ERROR,,0.4,0,0.00556109,3,2,238.15,0.000451262,43.2731,0.75,secrets/wandb.txt,True,iowa
my_xgb_853a50b2,ERROR,,0.55,0,0.0154611,0,1,5605.97,0.00272955,4.48938,0.7,secrets/wandb.txt,True,iowa
my_xgb_853d0ec4,ERROR,,0.5,0,0.00126552,5,3,4734.92,0.00967981,33.1173,0.65,secrets/wandb.txt,True,iowa
my_xgb_853f745c,ERROR,,0.75,0,0.00153686,2,2,678.644,0.306625,1.59856,0.9,secrets/wandb.txt,True,iowa


TuneError: ('Trials did not complete', [my_xgb_851c46f8, my_xgb_8523af6a, my_xgb_85286f14, my_xgb_852d4e3a, my_xgb_8531d554, my_xgb_8534f112, my_xgb_8537cbf8, my_xgb_853a50b2, my_xgb_853d0ec4, my_xgb_853f745c, my_xgb_85420488, my_xgb_8543b472, my_xgb_854602fe, my_xgb_8547fe88, my_xgb_854a4d32, my_xgb_854c45ba, my_xgb_854e3ff0, my_xgb_8550fec0, my_xgb_8552f81a, my_xgb_8555cd9c, my_xgb_8558f47c, my_xgb_855e2456, my_xgb_856185d8, my_xgb_85657f1c, my_xgb_856acaee, my_xgb_856feca4, my_xgb_85758e84, my_xgb_857ac8cc, my_xgb_857f3862, my_xgb_8583e056, my_xgb_85879a02, my_xgb_858c2c52, my_xgb_8590c776, my_xgb_8594c7c2, my_xgb_8598f040, my_xgb_859d2480, my_xgb_85a13642, my_xgb_85a5ae0c, my_xgb_85a9cbea, my_xgb_85ae3c8e, my_xgb_85b29d10, my_xgb_85b72fec, my_xgb_85bba8a6, my_xgb_85c075de, my_xgb_85c48d54, my_xgb_85c94f4c, my_xgb_85cdcbf8, my_xgb_85d28e90, my_xgb_85d7a722, my_xgb_85dd0f8c, my_xgb_85e154ca, my_xgb_85e57b2c, my_xgb_85e9d5e6, my_xgb_85ee8686, my_xgb_85f389c4, my_xgb_85f7eb22, my_xgb_8

In [ ]:
analysis.results_df.columns

In [ ]:
analysis_results_df = analysis.results_df[['mse', 'date', 'time_this_iter_s',
       'config.n_estimators', 'config.max_depth', 'config.min_child_weight', 'config.subsample',
       'config.colsample_bytree', 'config.reg_alpha', 'config.reg_lambda', 'config.gamma',
       'config.learning_rate']].sort_values('mse')
analysis_results_df


In [ ]:
max_depth = analysis_results_df.iloc[0]['config.max_depth']
min_child_weight = analysis_results_df.iloc[0]['config.min_child_weight']
subsample = analysis_results_df.iloc[0]['config.subsample']
colsample_bytree = analysis_results_df.iloc[0]['config.colsample_bytree']
reg_alpha = analysis_results_df.iloc[0]['config.reg_alpha']
reg_lambda = analysis_results_df.iloc[0]['config.reg_lambda']
reg_gamma = analysis_results_df.iloc[0]['config.gamma']
learning_rate = analysis_results_df.iloc[0]['config.learning_rate']
N_ESTIMATORS = analysis_results_df.iloc[0]['config.n_estimators']    


In [ ]:
best_config = {
    'max_depth': max_depth,
    'min_child_weight': min_child_weight,
    'subsample': subsample,
    'colsample_bytree': colsample_bytree,
    'reg_alpha': reg_alpha,
    'reg_lambda': reg_lambda,
    'gamma': reg_gamma,
    'learning_rate': learning_rate,
    'n_estimators':  N_ESTIMATORS
}

xgb = XGBRegressor(
    objective='reg:squarederror',
    random_state=RANDOMSTATE,    
    verbosity=1,
    n_jobs=-1,
    **best_config
)
print(xgb)

scores = -cross_val_score(xgb, df[predictors], df[response],
                          scoring="neg_root_mean_squared_error",
                          cv=kfolds)

raw_scores = [cv_to_raw(x) for x in scores]
print()
print("Log1p CV RMSE %.06f (STD %.04f)" % (np.mean(scores), np.std(scores)))
print("Raw CV RMSE %.06f (STD %.04f)" % (np.mean(raw_scores), np.std(raw_scores)))
raw_scores = [cv_to_raw(x) for x in scores]


## BayesOpt

In [ ]:
# bayesopt
NUM_SAMPLES=256

start_time = datetime.now()
print("%-20s %s" % ("Start Time", start_time))

algo = BayesOptSearch(utility_kwargs={
    "kind": "ucb",
    "kappa": 2.5,
    "xi": 0.0
})

# uncomment and set max_concurrent to limit number of cores
# algo = ConcurrencyLimiter(algo, max_concurrent=10)
scheduler = AsyncHyperBandScheduler()

tune_kwargs = {
    "num_samples": NUM_SAMPLES,
    "config": {
        "n_estimators": tune.loguniform(100, 10000),
        "max_depth": tune.quniform(0, 6, 1),
        'min_child_weight': tune.quniform(0, 6, 1),
        "subsample": tune.quniform(0.4, 0.9, 0.05),
        "colsample_bytree": tune.quniform(0.05, 0.8, 0.05),
        "reg_alpha": tune.loguniform(1e-04, 1),
        "reg_lambda": tune.loguniform(1e-04, 100),
        "gamma": 0,
        "learning_rate": tune.loguniform(0.001, 0.1),
        "wandb": {
            "project": "iowa",
            "api_key_file": "secrets/wandb.txt",
            "log_config": True
        }    
    }
}

analysis = tune.run(my_xgb,
                    name="xgb_bayesopt",
                    metric="mse",
                    mode="min",
                    search_alg=algo,
                    scheduler=scheduler,
                    verbose=1,
                    loggers=DEFAULT_LOGGERS + (WandbLogger, ),
                    **tune_kwargs)

end_time = datetime.now()
print("%-20s %s" % ("Start Time", start_time))
print("%-20s %s" % ("End Time", end_time))
print(str(timedelta(seconds=(end_time-start_time).seconds)))

In [ ]:
analysis_results_df = analysis.results_df[['mse', 'date', 'time_this_iter_s',
       'config.n_estimators', 'config.max_depth', 'config.min_child_weight', 'config.subsample',
       'config.colsample_bytree', 'config.reg_alpha', 'config.reg_lambda', 'config.gamma',
       'config.learning_rate']].sort_values('mse')
analysis_results_df

In [ ]:
max_depth = analysis_results_df.iloc[0]['config.max_depth']
min_child_weight = analysis_results_df.iloc[0]['config.min_child_weight']
subsample = analysis_results_df.iloc[0]['config.subsample']
colsample_bytree = analysis_results_df.iloc[0]['config.colsample_bytree']
reg_alpha = analysis_results_df.iloc[0]['config.reg_alpha']
reg_lambda = analysis_results_df.iloc[0]['config.reg_lambda']
reg_gamma = analysis_results_df.iloc[0]['config.gamma']
learning_rate = analysis_results_df.iloc[0]['config.learning_rate']
N_ESTIMATORS = analysis_results_df.iloc[0]['config.n_estimators']    

best_config = {
    'max_depth': max_depth,
    'min_child_weight': min_child_weight,
    'subsample': subsample,
    'colsample_bytree': colsample_bytree,
    'reg_alpha': reg_alpha,
    'reg_lambda': reg_lambda,
    'gamma': reg_gamma,
    'learning_rate': learning_rate,
    'n_estimators':  N_ESTIMATORS
}

xgb = XGBRegressor(
    objective='reg:squarederror',
    random_state=RANDOMSTATE,    
    verbosity=1,
    n_jobs=-1,
    **best_config
)
print(xgb)

scores = -cross_val_score(xgb, df[predictors], df[response],
                          scoring="neg_root_mean_squared_error",
                          cv=kfolds)

raw_scores = [cv_to_raw(x) for x in scores]
print()
print("Log1p CV RMSE %.06f (STD %.04f)" % (np.mean(scores), np.std(scores)))
print("Raw CV RMSE %.06f (STD %.04f)" % (np.mean(raw_scores), np.std(raw_scores)))


## Optuna

In [ ]:
# optuna
NUM_SAMPLES=256
optuna_xgb = my_xgb

start_time = datetime.now()
print("%-20s %s" % ("Start Time", start_time))

algo = OptunaSearch()
# uncomment and set max_concurrent to limit number of cores
# algo = ConcurrencyLimiter(algo, max_concurrent=10)
scheduler = AsyncHyperBandScheduler()

tune_kwargs = {
    "num_samples": NUM_SAMPLES,
    "config": {
        "n_estimators": tune.loguniform(100, 10000),
        "max_depth": tune.quniform(0, 6, 1),
        'min_child_weight': tune.quniform(0, 6, 1),
        "subsample": tune.quniform(0.4, 0.9, 0.05),
        "colsample_bytree": tune.quniform(0.05, 0.8, 0.05),
        "reg_alpha": tune.loguniform(1e-04, 1),
        "reg_lambda": tune.loguniform(1e-04, 100),
        "gamma": 0,
        "learning_rate": tune.loguniform(0.001, 0.1),
        "wandb": {
            "project": "iowa",
            "api_key_file": "secrets/wandb.txt",
            "log_config": True,
            "name": get_random_tag(6)
        }           
    }
}

analysis = tune.run(optuna_xgb,
                    name="xgb_optuna",
                    metric="mse",
                    mode="min",
                    search_alg=algo,
                    scheduler=scheduler,
                    verbose=1,
                    loggers=DEFAULT_LOGGERS + (WandbLogger, ),                    
                    **tune_kwargs)

end_time = datetime.now()
print("%-20s %s" % ("Start Time", start_time))
print("%-20s %s" % ("End Time", end_time))
print(str(timedelta(seconds=(end_time-start_time).seconds)))

In [ ]:
analysis_results_df = analysis.results_df[['mse', 'date', 'time_this_iter_s',
       'config.n_estimators', 'config.max_depth', 'config.min_child_weight', 'config.subsample',
       'config.colsample_bytree', 'config.reg_alpha', 'config.reg_lambda', 'config.gamma',
       'config.learning_rate']].sort_values('mse')
analysis_results_df

In [ ]:
max_depth = analysis_results_df.iloc[0]['config.max_depth']
min_child_weight = analysis_results_df.iloc[0]['config.min_child_weight']
subsample = analysis_results_df.iloc[0]['config.subsample']
colsample_bytree = analysis_results_df.iloc[0]['config.colsample_bytree']
reg_alpha = analysis_results_df.iloc[0]['config.reg_alpha']
reg_lambda = analysis_results_df.iloc[0]['config.reg_lambda']
reg_gamma = analysis_results_df.iloc[0]['config.gamma']
learning_rate = analysis_results_df.iloc[0]['config.learning_rate']
N_ESTIMATORS = analysis_results_df.iloc[0]['config.n_estimators']    

best_config = {
    'max_depth': max_depth,
    'min_child_weight': min_child_weight,
    'subsample': subsample,
    'colsample_bytree': colsample_bytree,
    'reg_alpha': reg_alpha,
    'reg_lambda': reg_lambda,
    'gamma': reg_gamma,
    'learning_rate': learning_rate,
    'n_estimators':  N_ESTIMATORS
}

xgb = XGBRegressor(
    objective='reg:squarederror',
    random_state=RANDOMSTATE,    
    verbosity=1,
    n_jobs=-1,
    **best_config
)
print(xgb)

scores = -cross_val_score(xgb, df[predictors], df[response],
                          scoring="neg_root_mean_squared_error",
                          cv=kfolds)

raw_scores = [cv_to_raw(x) for x in scores]
print()
print("Log1p CV RMSE %.06f (STD %.04f)" % (np.mean(scores), np.std(scores)))
print("Raw CV RMSE %.06f (STD %.04f)" % (np.mean(raw_scores), np.std(raw_scores)))
raw_scores = [cv_to_raw(x) for x in scores]

# LightGBM with Optuna

In [ ]:
# tune LightGBM
print("LightGBM")
#!conda install -y -c conda-forge lightgbm

NUM_SAMPLES=256

start_time = datetime.now()
print("%-20s %s" % ("Start Time", start_time))

def my_lgbm(config):
    
    # fix these configs 
    config['n_estimators'] = int(config['n_estimators'])   # pass float eg loguniform distribution, use int
    config['num_leaves'] = 2 + int(config['num_leaves'])
    
    lgbm = LGBMRegressor(objective='regression',
                         num_leaves=config['num_leaves'],
                         learning_rate=config['learning_rate'],
                         n_estimators=config['n_estimators'],
                         max_bin=200,
                         bagging_fraction=config['bagging_fraction'],
                         feature_fraction=config['feature_fraction'],
                         feature_fraction_seed=7,
                         min_data_in_leaf=2,
                         verbose=-1,
                         # early stopping params, maybe in fit
                         #early_stopping_rounds=early_stopping_rounds,
                         #valid_sets=[xgtrain, xgvalid], valid_names=['train','valid'], evals_result=evals_results
                         #num_boost_round=num_boost_round,
                         )
    
    scores = np.sqrt(-cross_val_score(lgbm, df[predictors], df[response],
                                      scoring="neg_mean_squared_error",
                                      cv=kfolds))
    tune.report(mse=np.mean(scores))
    return {'mse': np.mean(scores)}

algo = HyperOptSearch(random_state_seed=RANDOMSTATE)
# uncomment and set max_concurrent to limit number of cores
# algo = ConcurrencyLimiter(algo, max_concurrent=10)
scheduler = AsyncHyperBandScheduler()

tune_kwargs = {
    "num_samples": NUM_SAMPLES,
    "config": {
        "n_estimators": tune.loguniform(100, 10000),
        'num_leaves': tune.randint(0, 10),
        "bagging_fraction": tune.uniform(0.5, 0.8),
        "feature_fraction": tune.uniform(0.01, 0.8),
        "learning_rate": tune.loguniform(0.001, 0.1),
        "wandb": {
            "project": "iowa",
            "api_key_file": "secrets/wandb.txt",
            "log_config": True
        }    
    }
}

analysis = tune.run(my_lgbm,
                    name="lgbm_hyperopt",
                    metric="mse",
                    mode="min",
                    search_alg=algo,
                    scheduler=scheduler,
                    verbose=1,
                    loggers=DEFAULT_LOGGERS + (WandbLogger, ),
                    **tune_kwargs)

end_time = datetime.now()
print("%-20s %s" % ("Start Time", start_time))
print("%-20s %s" % ("End Time", end_time))
print(str(timedelta(seconds=(end_time-start_time).seconds)))


# Ray Cluster

- Look at `ray1.0.yaml`
- Edit `ray1.0.yaml` file with your region, availability zone, subnet, imageid information
    - to get those variables launch the latest Deep Learning AMI (Ubuntu 18.04) Version 35.0 into a small instance in your favorite region/zone
    - test that it works
    - note those 4 variables: region, availability zone, subnet, AMI imageid
    - terminate the instance and edit `ray1.0.yaml` accordingly
    - in future you can create use own image with everything pre-installed and specify its AMI imageid
- To run the cluster do 
`ray up ray1.0.yaml`
    - Creates head instance using image specified.
    - Installs ray and related requirements
    - Clones this Iowa repo
    - Launches worker nodes per auto-scaling parameters (currently start, min, max=4)
- After cluster starts note that several instances started in AWS console
- Check `ray monitor ray1.0.yaml` for any error messages
- Run Jupyter on cluster with port forwarding
 `ray exec ray1.0.yaml --port-forward=8899 'source ~/anaconda3/bin/activate tensorflow_p36 && jupyter notebook --port=8899'`
- Open the notebook on the generated URL e.g. http://localhost:8899/?token=5f46d4355ae7174524ba71f30ef3f0633a20b19a204b93b4
- Choose the default kernel to make sure it runs in the conda environment with all installs
- Use the specified ray.init() command
- You can also run a terminal on the head node of the cluster with
 `ray attach /Users/drucev/projects/iowa/ray1.0.yaml`
- You can also ssh explicitly with the IP address and the generated private key
 `ssh -o IdentitiesOnly=yes -i ~/.ssh/ray-autoscaler_1_us-east-1.pem ubuntu@54.161.200.54`
- run port forwarding to the Ray dashboard with   
`ray dashboard ray1.0.yaml`
and then open
 http://localhost:8265/

see https://docs.ray.io/en/latest/cluster/launcher.html for additional info